In [ ]:
# Import the libraries

import matplotlib.pyplot as plt
import numpy as np
from skimage import measure, io, img_as_ubyte
from skimage.color import rgb2gray, rgba2rgb
from skimage.filters import threshold_otsu

In [ ]:
def gray_img(img):
    """
    Takes an image as the argument, returns it in grey scale.
    
    Parameters
    ----------
    img: The western blot image in the directory
    
    Returns
    ----------
    Same image in grey scale
    
    """
    return img_as_ubyte(rgb2gray(rgba2rgb(io.imread(img))))

In [ ]:
def thresh_img(grey_img):
    """
    Takes an grey scale image, returns threshold image with Otsu algorithm.
    
    Parameters
    ----------
    grey_img: the grey scale image of western blot provided by gray_img function
    
    Returns
    ----------
    Thresholded image from the grey scale image provided.
    
    """
    
    segmented = grey_img < threshold_otsu(grey_img)
    return segmented